<a href="https://colab.research.google.com/github/priyal6/NLP-Prac/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [35]:
from transformers import BertTokenizer, BertForSequenceClassification, PreTrainedModel
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import TrainingArguments, Trainer
from transformers import BertModel
from transformers.modeling_outputs import SequenceClassifierOutput

In [2]:
dataset = load_dataset("imdb", split = 'train[:2000]')
dataset = dataset.train_test_split(test_size=0.2)
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
val_texts = dataset['test']['text']
val_labels = dataset['test']['label']

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
def tokenize(batch):
  return tokenizer(batch['text'], padding = True, truncation= True, return_tensors='pt')

In [9]:
dataset = dataset[0].map(lambda x: tokenizer(x['text'], padding="max_length", truncation=True), batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [34]:
# Load pre-trained model with new output head
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2  # For binary classification
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
for param in model.bert.parameters():
  print(len(param))

30522
512
2
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768
768
768
768
768
768
768
768
768
3072
3072
768
768
768
768
768
768


In [17]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 109483778


In [18]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 109483778


In [19]:
for name, param in model.named_parameters():
  print(f"{name:50} | Shape: {tuple(param.shape)} | Trainable: {param.requires_grad}")

bert.embeddings.word_embeddings.weight             | Shape: (30522, 768) | Trainable: True
bert.embeddings.position_embeddings.weight         | Shape: (512, 768) | Trainable: True
bert.embeddings.token_type_embeddings.weight       | Shape: (2, 768) | Trainable: True
bert.embeddings.LayerNorm.weight                   | Shape: (768,) | Trainable: True
bert.embeddings.LayerNorm.bias                     | Shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.self.query.weight   | Shape: (768, 768) | Trainable: True
bert.encoder.layer.0.attention.self.query.bias     | Shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.self.key.weight     | Shape: (768, 768) | Trainable: True
bert.encoder.layer.0.attention.self.key.bias       | Shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.self.value.weight   | Shape: (768, 768) | Trainable: True
bert.encoder.layer.0.attention.self.value.bias     | Shape: (768,) | Trainable: True
bert.encoder.layer.0.attention.output.den

In [20]:
print(model.bert.embeddings)
print(model.bert.encoder.layer[11])
print(model.classifier)

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): 

In [21]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
print(model.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [23]:
training_args = TrainingArguments(
    output_dir="./bert_output_1",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
     report_to="none"
)

In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [25]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset['train'],
    eval_dataset= dataset['test'],
)

In [26]:
trainer.train()

Step,Training Loss
500,0.004100


TrainOutput(global_step=600, training_loss=0.003445415881772836, metrics={'train_runtime': 499.0685, 'train_samples_per_second': 9.618, 'train_steps_per_second': 1.202, 'total_flos': 1262933065728000.0, 'train_loss': 0.003445415881772836, 'epoch': 3.0})

In [29]:
from transformers import BertModel, BertTokenizer, PreTrainedModel, Trainer, TrainingArguments, BertConfig
from transformers.modeling_outputs import SequenceClassifierOutput

In [39]:
class BertBinaryClassifier(PreTrainedModel):
  def __init__(self, config):
    super().__init__(config)
    self.bert = BertModel.from_pretrained("bert-base-uncased")
    self.classifier = nn.Linear(config.hidden_size,2 )

    for param in self.bert.parameters():
      param.requires_grad = False

    for layer in self.bert.encoder.layer[-2:]:
      for param in layer.parameters():
        param.requires_grad = True

    for param in self.bert.pooler.parameters():
      param.requires_grad = True

    for param in self.classifier.parameters():
      param.requires_grad=True


  def forward(self, input_ids, attention_mask, labels =None):
    outputs = self.bert(input_ids = input_ids, attention_mask = attention_mask)
    pooled_output = outputs.pooler_output
    logits = self.classifier(pooled_output)

    loss = None

    if labels is not None:
      loss_fn = nn.CrossEntropyLoss()
      loss = loss_fn(logits, labels)

    return SequenceClassifierOutput(
              loss=loss,
              logits=logits
          )

In [40]:
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2)
model = BertBinaryClassifier(config)

training_args = TrainingArguments(
    output_dir="./bert_last2layers_finetune",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="no",
    report_to="none",  # Disable wandb etc.
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()

Step,Training Loss
500,0.003800


TrainOutput(global_step=600, training_loss=0.0031477145377236107, metrics={'train_runtime': 199.0171, 'train_samples_per_second': 24.119, 'train_steps_per_second': 3.015, 'total_flos': 1262933065728000.0, 'train_loss': 0.0031477145377236107, 'epoch': 3.0})